##  Purpose: Build CNN model in Functional API for compatibility with Grad-CAM + Gradio

# data_preparation

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Flatten

# Set a global random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load Fashion MNIST dataset directly from Keras
# It returns: (X_train, y_train), (X_test, y_test)
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Normalize pixel values to range [0, 1] by dividing by 255
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

# Reshape input data to add a channel dimension (since images are grayscale)
# Shape becomes (num_samples, 28, 28, 1) for CNN compatibility
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# Print dataset shapes for confirmation
print(f"[INFO] X_train shape: {X_train.shape}")
print(f"[INFO] X_test shape: {X_test.shape}")
print(f"[INFO] y_train shape: {y_train.shape}")
print(f"[INFO] y_test shape: {y_test.shape}")

[INFO] X_train shape: (60000, 28, 28, 1)
[INFO] X_test shape: (10000, 28, 28, 1)
[INFO] y_train shape: (60000,)
[INFO] y_test shape: (10000,)


# defining the model that will be compatibel with gradio

In [13]:
# -----------------------------
# Define input shape explicitly
# -----------------------------
# This ensures the model has a defined input tensor, unlike Sequential models where .input might be undefined unless called
input_layer = Input(shape=(28, 28, 1), name="input_layer")

# -----------------------------
# First Convolutional Block
# -----------------------------
# Conv2D + BatchNorm + MaxPooling + Dropout for regularization and feature extraction
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

# -----------------------------
# Second Convolutional Block
# -----------------------------
# More filters to extract complex patterns; layer named for Grad-CAM targeting
x = Conv2D(64, (3, 3), activation='relu', padding='same', name="conv2d_1")(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

# -----------------------------
# Third Convolutional Block
# -----------------------------
# Deeper representation of image; this conv layer is often used for Grad-CAM
x = Conv2D(128, (3, 3), activation='relu', padding='same', name="conv2d_2")(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

# -----------------------------
# Fully Connected Layers
# -----------------------------
# Flatten to vector, then use Dense layers for classification
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(10, activation='softmax')(x)  # 10 classes for Fashion MNIST

# -----------------------------
# Build Model using Functional API
# -----------------------------
# This gives explicit access to model.input and model.output, required for Grad-CAM
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# -----------------------------
# Trigger model building
# -----------------------------
# Functional models need to be 'called' before .output/.input is defined
_ = model.predict(np.zeros((1, 28, 28, 1)))  # Dummy call

# -----------------------------
# Save model for use in Stage 6 (Grad-CAM + Gradio)
# -----------------------------


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step


# trainig and fine tubning

In [14]:

# 12. Train the model
# ------------------------------------------------------------
# Fit the model for 10 epochs with validation
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=64,
    verbose=2
)

# ------------------------------------------------------------
# 13. Evaluate and report accuracy
# ------------------------------------------------------------
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"[RESULT] Test Accuracy: {test_accuracy:.4f}")
print(f"[RESULT] Test Loss: {test_loss:.4f}")

# ------------------------------------------------------------
# 14. Save the trained model
# ------------------------------------------------------------
model.save("models/clothing_recognition_deep_cnn_gradio_modelbuilder.h5")
print("[INFO] Trained model saved (Grad-CAM + Gradio ready)")

Epoch 1/10
938/938 - 16s - 17ms/step - accuracy: 0.7645 - loss: 0.6594 - val_accuracy: 0.8637 - val_loss: 0.3762
Epoch 2/10
938/938 - 4s - 4ms/step - accuracy: 0.8496 - loss: 0.4180 - val_accuracy: 0.8850 - val_loss: 0.3139
Epoch 3/10
938/938 - 4s - 4ms/step - accuracy: 0.8690 - loss: 0.3599 - val_accuracy: 0.8785 - val_loss: 0.3458
Epoch 4/10
938/938 - 4s - 4ms/step - accuracy: 0.8785 - loss: 0.3344 - val_accuracy: 0.8585 - val_loss: 0.4110
Epoch 5/10
938/938 - 4s - 4ms/step - accuracy: 0.8862 - loss: 0.3135 - val_accuracy: 0.9072 - val_loss: 0.2576
Epoch 6/10
938/938 - 4s - 4ms/step - accuracy: 0.8921 - loss: 0.2991 - val_accuracy: 0.8697 - val_loss: 0.3737
Epoch 7/10
938/938 - 4s - 4ms/step - accuracy: 0.8955 - loss: 0.2878 - val_accuracy: 0.8956 - val_loss: 0.2944
Epoch 8/10
938/938 - 4s - 4ms/step - accuracy: 0.9009 - loss: 0.2772 - val_accuracy: 0.9085 - val_loss: 0.2551
Epoch 9/10
938/938 - 4s - 4ms/step - accuracy: 0.9024 - loss: 0.2689 - val_accuracy: 0.9159 - val_loss: 0.2369

[RESULT] Test Accuracy: 0.9158
[RESULT] Test Loss: 0.2424
[INFO] Trained model saved (Grad-CAM + Gradio ready)


In [15]:
from google.colab import files
files.download("models/clothing_recognition_deep_cnn_gradio_modelbuilder.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>